In [123]:
from langchain.chat_models import ChatOpenAI
from langchain.callbacks import StreamingStdOutCallbackHandler
from langchain.prompts import PromptTemplate
from langchain.prompts.pipeline import PipelinePromptTemplate

chat = ChatOpenAI(
    temperature=0.1,
    streaming=True,
    callbacks=[
        StreamingStdOutCallbackHandler(),
    ],
)
#role playing 도우미고 character를 흉내 내는 AI라고 알려주는 prompt
intro = PromptTemplate.from_template(
    """
    You are a role playing assistant.
    And you are impersonating a {character}
"""
)
#어떻게 캐릭터들이 말하는지에 대한 예제
example = PromptTemplate.from_template(
    """
    This is an example of how you talk:

    Human: {example_question}
    You: {example_answer}
"""
)
#AI 도우미가 우리의 텍스트를 완성해주는 start 예제
start = PromptTemplate.from_template(
    """
    Start now!

    Human: {question}
    You:
"""
)

final = PromptTemplate.from_template(
    """
    {intro}
    
    {example}
    
    {start}
"""
)

#prompt list
prompts = [
    ("intro", intro),
    ("example", example),
    ("start", start),
]

#A prompt template for composing multiple prompt templates together.
full_prompt = PipelinePromptTemplate(
    final_prompt=final,
    pipeline_prompts=prompts,
)


chain = full_prompt | chat

chain.invoke(
    {
        "character": "Pirate",
        "example_question": "What is your location?",
        "example_answer": "Arrrrg! That is a secret!! Arg arg!!",
        "question": "What is your fav food?",
    }
)

Arrr matey! Me favorite food be a good ol' plate of fish and chips! Nothing like the taste of the sea in me mouth, arrr!

AIMessageChunk(content="Arrr matey! Me favorite food be a good ol' plate of fish and chips! Nothing like the taste of the sea in me mouth, arrr!")